# Small Datasets - Multi-Channel Timeseries with Numpy

TODO create banner image
![]()

TODO: find and use a real EMG or EKG dataset

---

## Overview

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold"> Visit the Index Page </p>
    This workflow example is part of set of related workflows. If you haven't already, visit the <a href="/index.html">index</a> page for an introduction and guidance on choosing the appropriate workflow.
</div>

The intended use-case for this workflow is to browse and annotate multi-channel timeseries data from an [electrophysiological](https://en.wikipedia.org/wiki/Electrophysiology) recording session.

TODO: write overview specific to smaller dataset situations

## Prerequisites and Resources

| Topic | Type | Notes |
| --- | --- | --- |
| [Intro and Guidance](./index.ipynb) | Prerequisite | Background |
| [Time Range Annotation](./time_range_annotation.ipynb) | Next Step | Display and edit time ranges |
| [Medium Dataset Workflow](./medium_multi-chan-ts.ipynb) | Alternative | Use Pandas and downsample |
| [Larger Dataset Workflow](./large_multi-chan-ts.ipynb) | Alternative | Use dynamic data chunking |

---

## Imports and Configuration

In [ ]:
import numpy as np

import colorcet as cc
import holoviews as hv
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from bokeh.models import HoverTool
import panel as pn

pn.extension()
hv.extension('bokeh')
np.random.seed(0)

## Generate a Small Fake Dataset

TODO: replace with a small real EMG dataset

In [ ]:
n_channels = 6
n_seconds = 300
sampling_rate = 128

initial_frequency = .01
frequency_increment = 2/n_channels
amplitude = 1

total_samples = n_seconds * sampling_rate
time = np.linspace(0, n_seconds, total_samples)

# Let's just name our channels 'CH 0', 'CH 1', ...
channels = [f'CH {i}' for i in range(n_channels)]

# We'll also add a grouping to our channels
groups = ['EEG'] * (n_channels // 2) + ['MEG'] * (n_channels - n_channels // 2)

data = np.array([amplitude * np.sin(2 * np.pi * (initial_frequency + i * frequency_increment) * time)
                 for i in range(n_channels)])

print(f'shape: {data.shape} (n_channels, samples) ')

## Visualize multi-channel timeseries

In [ ]:
# TODO: different groups would have different units so need to change the amplitude dim

In [ ]:

time_dim = hv.Dimension('Time', unit='s')
amplitude_dim = hv.Dimension('Amplitude', unit='µV')

# Create curves overlay plot
curves = []
for group, channel, channel_data in zip(groups, channels, data):
    ds = hv.Dataset((time, channel_data), [time_dim, amplitude_dim])
    curve = hv.Curve(ds, time_dim, amplitude_dim, group=group, label=f'{channel}')
    curve.opts(
        subcoordinate_y=True,
        subcoordinate_scale=.75,
        color="black",
        line_width=1,
        tools=['hover'],
        hover_tooltips=[("Type", "$group"), ("Channel", "$label"), "Time", "Amplitude"],
        )
    curves.append(curve)

curves_overlay = hv.Overlay(curves, kdims="Channel")

# set opts on overlay, including group-wise coloring
color_map = dict(zip(set(groups), cc.b_glasbey_bw_minc_20[::-1][:len(set(groups))]))
group_color_opts = [hv.opts.Curve(grp, color=grpclr) for grp, grpclr in color_map.items()]
curves_overlay = curves_overlay.opts(
    *group_color_opts,
    hv.opts.Overlay(
    xlabel="Time (s)", ylabel="Channel", show_legend=False,
    padding=0, aspect=1.5, responsive=True, shared_axes=False, framewise=False, min_height=100,)
)

# Create minimap
y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)
minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel='',
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    toolbar='disable',
    height=120,
    responsive=True,
    default_tools=[],
    )

# Link minimap widget to curves overlay plot
RangeToolLink(minimap, curves_overlay, axes=["x", "y"],
              boundsx=(0, time[len(time)//3]) # initial range of the minimap
             )

app = pn.Column((curves_overlay + minimap).cols(1), min_height=500).servable()
app


## Complete Code for Application

In [ ]:
n_channels = 6
n_seconds = 300
sampling_rate = 128

initial_frequency = .01
frequency_increment = 2/n_channels
amplitude = 1

total_samples = n_seconds * sampling_rate
time = np.linspace(0, n_seconds, total_samples)

channels = [f'CH {i}' for i in range(n_channels)]
groups = ['EEG'] * (n_channels // 2) + ['MEG'] * (n_channels - n_channels // 2)
data = np.array([amplitude * np.sin(2 * np.pi * (initial_frequency + i * frequency_increment) * time)
                 for i in range(n_channels)])

time_dim = hv.Dimension('Time', unit='s')
amplitude_dim = hv.Dimension('Amplitude', unit='µV')

# set group colors
color_map = dict(zip(set(groups), cc.b_glasbey_bw_minc_20[::-1][:len(set(groups))]))
group_color_opts = [hv.opts.Curve(grp, color=grpclr) for grp, grpclr in color_map.items()]

# Create curves overlay plot
curves = []
for group, channel, channel_data in zip(groups, channels, data):
    ds = hv.Dataset((time, channel_data), [time_dim, amplitude_dim])
    curve = hv.Curve(ds, time_dim, amplitude_dim, group=group, label=f'{channel}')
    curve.opts(
        subcoordinate_y=True,
        subcoordinate_scale=.75,
        color="black",
        line_width=1,
        tools=['hover'],
        hover_tooltips=[("Group", "$group"), ("Channel", "$label"), "Time", "Amplitude"],
        )
    curves.append(curve)

curves_overlay = hv.Overlay(curves, "Channel")

curves_overlay = curves_overlay.opts(
    *group_color_opts,
    hv.opts.Overlay(
    xlabel="Time (s)", ylabel="Channel", show_legend=False,
    padding=0, aspect=1.5, responsive=True, shared_axes=False, framewise=False, min_height=100,)
)

# Create minimap
y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel='',
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    toolbar='disable',
    height=120,
    responsive=True,
    default_tools=[],
    )

# Link minimap widget to curves overlay plot
RangeToolLink(minimap, curves_overlay, axes=["x", "y"],
              boundsy=(-.5, 5.5),
              boundsx=(0, time[len(time)//3])
             )

app = pn.Column((curves_overlay + minimap).cols(1), min_height=500).servable()
app


## Summary

### What's next?